# pandas 入门

pandas 主要功能: 

* 具备按轴自动或显示数据对齐功能的数据结构.
* 集成时间序列功能.
* 即能处理时间序列数据也能处理非时间序列数据的数据结构.
* 数学运算和约简(比如对某个轴求和) 可以根据不同的元数据执行.
* 灵活处理缺失数据.
* 合并及其他出现在常见数据库中的关系型运算.

In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

## Series

Series 是一种类似于一维数组的对象，它由一组数据(各种NumPy数据类型)以及一组与之相关的数据标签(即索引)组成。仅由一组数据即可产生最简单的 Series :

In [2]:
obj = Series([4, 7, -5, 3])
obj

0    4
1    7
2   -5
3    3
dtype: int64

Series 的字符串表现形式为: 索引在左边，值在右边。由于我们没有为数据指定索引，于是会自动创建一个 0 到 N1(N 为数据的长度) 的整数型索引, 可以通过 Series 的 values 和 index 属性获取其数组表示形式和索引对象:

In [3]:
obj.values

array([ 4,  7, -5,  3])

In [4]:
obj.index

RangeIndex(start=0, stop=4, step=1)

In [5]:
# 创建 Series 带有一个可以对各个数据点进行标记的索引
obj2 = Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])
obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [6]:
obj2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

In [7]:
# 与普通的 NumPy 数组项目可通过索引方式选取单个或一组值
obj2['a']

-5

In [8]:
obj2['d'] = 6

In [9]:
obj2[ ['c', 'a', 'd'] ]

c    3
a   -5
d    6
dtype: int64

NumPy 数组运算(如根据布尔型数组进行过滤、标量乘法、应用数学函数等)都会保留索引和值之间的链接:

In [10]:
obj2

d    6
b    7
a   -5
c    3
dtype: int64

In [11]:
obj2[obj2 > 0]

d    6
b    7
c    3
dtype: int64

In [12]:
obj2 * 2

d    12
b    14
a   -10
c     6
dtype: int64

In [13]:
np.exp(obj2)

d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64

可以将 Series 看成是一个定长的有序字典，因为它是索引值到数据值的一个映射.

In [14]:
'b' in obj2

True

In [15]:
'e' in obj2

False

通过字典来创建 Series, 没有给定 index 的时候，默认使用字典的键，并根据键排列 :

In [16]:
sdata = {
    'Ohio': 35000,
    'Texas': 71000,
    'Oregon': 16000,
    'Utah': 5000
}
obj3 = Series(sdata)
obj3

Ohio      35000
Oregon    16000
Texas     71000
Utah       5000
dtype: int64

In [17]:
# 指定索引
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = Series(sdata, index=states)
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

上面例子中 sdata 中跟 states 索引相匹配的那个 3 个值会被找出来并放到相应的位置上，但由于 'California' 所对应的 sdata 值找不到，所以其结果就是为 NaN, pandas 的 isnull 和 notnull 函数可以用于检测缺失数据：

In [18]:
pd.isnull(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [19]:
pd.notnull(obj4)

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

In [20]:
# 同样有实例方法
obj4.isnull()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

Series 最重要的一个功能是: 它在算术运算中会自动对齐不同索引的数据。

In [21]:
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

Series 对象本身及其索引都有一个 name 属性, 该属性跟 pandas 其他的关键功能关系非常密切:

In [22]:
obj4.name = 'population'
obj4.index.name = 'state'
obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

In [23]:
# 就地修改 idnex 
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

## DataFrame

DataFrame 是一个表格型的数据结构，它含有一组有序的列，每列可以是不同的值类型(数值、字符串、布尔值等)。DataFrame 即有行索引也有列索引，它可以被看作由 Series 组成的字典(共用一个索引)

构建 DataFrame 的办法有很多，最常用的一种是直接传入一个由等长列表或NumPy数组组成的字典

In [24]:
data = {
    'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'], 
    'year': [2000, 2001, 2002, 2001, 2002],
    'pop': [1.5, 1.7, 3.6, 2.4, 2.9]
}
frame = DataFrame(data)
frame

,pop,state,year
0,1.5,Ohio,2000
1,1.7,Ohio,2001
2,3.6,Ohio,2002
3,2.4,Nevada,2001
4,2.9,Nevada,2002


如果指定了列序列，则 DataFrame 的列就会按照指定顺序进行排列

In [25]:
DataFrame(data, columns=['year', 'state', 'pop'])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9


和 Series 一样，如果传入的列在数据中找不到，就会产生 NA 值

In [26]:
frame2 = DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
                   index=['one', 'two', 'three', 'four', 'five'])
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN


In [27]:
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

通过类似字典标记的方式或属性的方式，可以将 DataFrame 的列获取为一个 Series

In [28]:
frame['state']

0      Ohio
1      Ohio
2      Ohio
3    Nevada
4    Nevada
Name: state, dtype: object

In [29]:
frame2.year

one      2000
two      2001
three    2002
four     2001
five     2002
Name: year, dtype: int64

Tip: 返回的 Series 拥有原 DataFrame 相同的索引，且 name 属性也已经被相应地设置好了。

In [30]:
frame2.loc['three']

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

列可以通过赋值的方式进行修改

In [31]:
frame2['debt'] = 16.5
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,16.5
two,2001,Ohio,1.7,16.5
three,2002,Ohio,3.6,16.5
four,2001,Nevada,2.4,16.5
five,2002,Nevada,2.9,16.5


In [32]:
frame2['debt'] = np.arange(5.)
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,0.0
two,2001,Ohio,1.7,1.0
three,2002,Ohio,3.6,2.0
four,2001,Nevada,2.4,3.0
five,2002,Nevada,2.9,4.0


将列表或数组赋值给某个列时，其长度必须跟 DataFrame 的长度相匹配。如果赋值的是一个 Series,就会精确匹配 DataFrame 的索引，所有空位都将被填上缺失值:

In [33]:
val = Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five'])
frame2['debt'] = val
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7


为不存在的列赋值会创建出一个新列， 关键字 del 用于删除列:

In [34]:
frame2['eastern'] = frame2.state == 'Ohio'
frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,-1.7,False


In [35]:
del frame2['eastern']
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

另一种常见的数据形式是嵌套字典, 外层字典的键作为列， 内层字典的键作为行索引

In [36]:
pop = {
    'Nevada': {2001: 2.4, 2002: 2.9},
    'Ohio': {2000: 1.5, 2001: 1.7, 2002: 2.6}
}
frame3 = DataFrame(pop)
frame3

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,2.6


内层字典的键会被合并、排序以形成最终索引，如果显示指定了索引，则不会这样:

In [37]:
DataFrame(pop, index=[2001, 2002, 2003])

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,2.6
2003,NaN,NaN


由 Series 组成的字典差不多也是一样的用法:

In [38]:
pdata = {
    'Ohio': frame3['Ohio'][: -1],
    'Nevada': frame3['Nevada'][: 2],
}
DataFrame(pdata)

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7


可以输入给 DataFrame 构造器的数据

| 类型 | 说明 |
| :--: | :--: |
| 二维ndarray | 数据矩阵，还可以传入行标和列标 |
| 由数组、列表或元组组成的字典 | 每个序列会变成 DataFrame 的一列。所有序列长度必须相同 |
| NumPy 的结构化/记录数组 | 类似于 "由数组组成的字典" |
| 由 Series 组成的字典 | 每个 Series 会成为一列。如果没有显式指定索引，则各 Series 的索引会被合并成结果的行索引 |
| 由字典组成的字典 | 各内层字典会成为一列。键会被合并成结果的行索引，跟 "由 Series 组成的字典" 的情况一样 |
| 字典或 Series 的列表 | 各项将会成为 DataFrame 的一行。 字典键或 Series 索引的并集将会成为 DataFrame 的列标 |
| 由列表或元组组成的列表 | 类似 "二维 ndarray" |
| 另一个 DataFrame | 该 DataFrame 的索引将会被沿用，除非显式指定了其他索引 |
| NumPy 的 MaskedArray | 类似于 "二维 ndarray" 的情况， 只是掩码值在结果 DataFrame 会变成NA/缺失值 |

如果设置了 DataFrame 的 index 和 columns 的 name 属性，则这些信息也被显示出来:

In [40]:
frame3.index.name = 'year'; frame3.columns.name = 'state'; frame3

state,Nevada,Ohio
year,,
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,2.6


和 Series 一样，values 属性也会以二维 ndarray 的形式返回 DataFrame 中的数据

In [41]:
frame3.values

array([[nan, 1.5],
       [2.4, 1.7],
       [2.9, 2.6]])

如果 DataFrame 各列的数据类型不同，则值数组的数据类型就会选用能兼容所有列的数据类型

In [42]:
frame2.values

array([[2000, 'Ohio', 1.5, nan],
       [2001, 'Ohio', 1.7, -1.2],
       [2002, 'Ohio', 3.6, nan],
       [2001, 'Nevada', 2.4, -1.5],
       [2002, 'Nevada', 2.9, -1.7]], dtype=object)

### 索引对象

pandas 的索引对象负责管理轴标签和其他元数据。构建 Series 或 DataFrame 时，所用到的任何数组或其他序列的标签都会被转换成一个 Index:

In [44]:
obj = Series(range(3), index=['a', 'b', 'c'])
index = obj.index
index

Index(['a', 'b', 'c'], dtype='object')

In [45]:
index[1:]

Index(['b', 'c'], dtype='object')

Index 对象是不可以修改的, 这样才能使 Index 对象在多个数据结构直接安全共享

In [46]:
index[1] = 'd'

TypeError: Index does not support mutable operations

In [47]:
index = pd.Index(np.arange(3))
obj2 = Series([1.5, -2.5, 0], index=index)

In [48]:
obj2.index is index

True

Index 的方法和属性

 | 方法 | 说明 |
 | :--: | :--: |
 | append | 连接另一个 Index 对象，产生一个新的 Index |
 | diff | 计算差集，并得到一个 Index |
 | intersection | 计算交集 |
 | union | 计算并集 |
 | isin | 计算一个指标各值是否都包含在参数集合中的布尔型数组 |
 | delete | 删除索引 i 处的元素，并得到新的 Index |
 | drop | 删除传入的值， 并得到新的 Index |
 | insert | 将元素插入到索引 i 处， 并得到新的 Index |
 | is_monotonic | 当各元素均大于等于前一个元素时，返回 True |
 | is_unique | 当 Index 没有重复值时， 返回 True |
 | unique | 计算 Index 中唯一值的数组 |

## 基本功能

## 重新索引

pandas 对象的重要方法 reindex, 其作用是创建一个适应新索引的新对象。

In [49]:
obj = Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

调用 reindex 将会格局新索引进行重排，如果某个索引值打过去不存在就会引入缺失值

In [50]:
obj2 = obj.reindex(list('abcde'))
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

In [52]:
obj.reindex(list('abcde'), fill_value=0) # 指定填充值

a   -5.3
b    7.2
c    3.6
d    4.5
e    0.0
dtype: float64

In [53]:
# 可以指定填充方式
obj3 = Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3

0      blue
2    purple
4    yellow
dtype: object

In [54]:
obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

reindex 的 method 选项

| 参数 | 说明 |
| :--: | :--: |
| ffill 或 pad | 向前填充值 |
| bfill 或 backfill | 向后填充 |

python3 的还有就近填充等

DataFrame 用 reindex 和  columns 修改行列索引

### 丢弃指定轴上的项

由于需要执行一些数据整理和集合逻辑，所以 drop 方法返回的是一个在指定轴山删除了指定值得新对象:

In [56]:
obj = Series(np.arange(5.), index=list('abcde'))
new_obj = obj.drop('c')

In [57]:
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [58]:
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [59]:
obj.drop(['d', 'c'])

a    0.0
b    1.0
e    4.0
dtype: float64

In [61]:
# 删除列
data = DataFrame(np.arange(16).reshape((4, 4)),
                 index=['Ohio', 'Colorado', 'Utah', 'New York'],
                 columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [62]:
# 通过指定 axis = 1 来删除列
data.drop('two', axis=1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


### 索引、选取和过滤

Series 索引的工作方式类似于 NunPy 数组的索引，只不过 Series 的索引值不只是整数

In [65]:
obj = Series(np.arange(4.), index=list('abcd'))
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [67]:
obj['b']

1.0

In [68]:
obj[1]

1.0

In [69]:
obj[2: 4]

c    2.0
d    3.0
dtype: float64

In [70]:
obj[['b', 'a', 'd']]

b    1.0
a    0.0
d    3.0
dtype: float64

In [71]:
obj[[1, 3]]

b    1.0
d    3.0
dtype: float64

In [72]:
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

Tip: 利用标签的切片运算和普通的 Python 切片运算不同，其末端是包含的

In [74]:
obj['b':'c']

b    1.0
c    2.0
dtype: float64

In [75]:
# 设置
obj['b': 'c'] = 5
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

对 DataFrame 进行索引其实就是获取一个或多个列

In [76]:
data = DataFrame(np.arange(16).reshape((4, 4)),
                 index=['Ohio', 'Colorado', 'Utah', 'New York'],
                 columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [77]:
data['two']

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int64

In [78]:
data[['three', 'one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


这种索引有几个特殊的情况，首先通过切片或布尔型数组选取行

In [79]:
data[: 2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [80]:
data[data['three'] > 5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


看起来这种用法不太符合逻辑，但这种语法的确切来源于实践

In [81]:
# 通过布尔型选择
data < 5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [82]:
data[ data < 5 ] = 0
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [85]:
data.ix[['Colorado', 'Utah'], [3, 0, 1]] # ix 方式在 python3 中已经用 loc 和 iloc 替代

,four,one,two
Colorado,7,0,5
Utah,11,8,9


### 算术运算和数据对齐

pandas 最重要的一个功能是，它可以对不同索引的对象进行算术运算，在将对象相加时，如果存在不同的索引对，则结果的索引就是该索引对的并集。

In [87]:
s1 = Series([7.3, -2.5, 3.4, 1.5], index=list('acde'))
s2 = Series([-2.1, 3.6, -1.5, 4, 3.1], index=list('acefg'))
s1

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64

In [88]:
s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

In [89]:
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

自动的数据对齐操作在不重叠的索引处引入 NA 值，缺失值会在算术运算过程中传播。

对于 DataFrame, 对齐操作会同时发生在行和列上:

In [96]:
df1 = DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),
                index=['Ohio', 'Texas', 'Colorado'])
df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [99]:
df2 = DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df2

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [100]:
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN
